In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN, KMeans

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cs_df = pd.read_csv('drive/MyDrive/allout.csv')

In [83]:
#Убираем строки, в которых документы с ошибкой
cs_df = cs_df.loc[cs_df['meta3'] < 400.0]
cs_df = cs_df.dropna()
cs_df.shape\
#Убираем пустые строки
cs_df = cs_df[~(cs_df.vector.isnull())]
#Убираем ненужные символы
cs_df['vector'] = cs_df['vector'].astype('string')
cs_df['vector'] = cs_df['vector'].map(lambda x: re.sub('[^A-Za-z0-9:\/\"\':;=]+',"", x))
cs_df['vector'].replace("", np.nan, inplace=True)
cs_df = cs_df.dropna()
cs_df.shape 


(5520, 7)

In [84]:
cs_df

,Unnamed: 0,Unnamed: 0.1,id,meta1,meta2,meta3,vector
1,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304.0,dlattach;topic=41020;attach=7045;image
5,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200.0,MPWM8At5
6,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200.0,/imgbasa/17896173683jpg
7,7,7,950ebbb818650193f9328d75453e69cb,REQUEST_HEADERS,Referer,200.0,file:///Applications/Install/639E2536C2764671B...
9,9,9,d2734f4ab17d4c4cd638f52a8628402a,REQUEST_HEADERS,Referer,200.0,file:///Applications/Install/65621A5D0DDE45B1A...
...,...,...,...,...,...,...,...
10043,998,991,5fd871e985fa1b628f22ec4d2aa45c31,REQUEST_GET_ARGS,url,302.0,http://taskalfacoza/mw/indexphptitle=8Mangaapp...
10045,1000,993,2f0f2a89b9742042d11bee029fdb9d3b,REQUEST_HEADERS,REQUEST_HEADERS.X-Forwarded-For,200.0,91185899
10046,1001,994,176ad9ac69bb3e8379ef6ebe17274165,REQUEST_GET_ARGS,ELEMENT_ID,200.0,33635'UNIONALLSELECTNULLNULLNULLCONCAT0x716a70...
10048,1003,996,8c881888c7512fe84d5258c80f580376,REQUEST_GET_ARGS,ELEMENT_ID,200.0,33635UNIONALLSELECTNULLNULLNULLNULLNULLNULLNUL...


In [85]:
#Кластеризация с помощью KMean
samples =cs_df['vector'].tolist()
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3))
vectors = vectorizer.fit_transform(samples)


clustering = KMeans(n_clusters=2, random_state=0).fit_predict(vectors)
new_df = pd.DataFrame(data=clustering, columns=['label'])
set(clustering)



{0, 1}

Получили два кластера - потенциально опасные и безопасные

In [86]:
#Визуализация
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(vectors)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [87]:
new_df["x_component"]=X_embedded[:,0]
new_df["y_component"]=X_embedded[:,1]

In [88]:
import plotly.express as px
    
fig = px.scatter(new_df, x="x_component", y="y_component", color = "label", size_max=60)
fig.update_layout(
     height=800)
fig.show()


In [144]:
#Посмотрим что в кластерах
foo = new_df.to_numpy()
bar = cs_df.to_numpy()
new_df = np.concatenate((foo, bar), axis=1)
new_df = pd.DataFrame(new_df)

dangerous_cluster = new_df.loc[new_df[0] == 1]
safe_cluster = new_df.loc[new_df[0] == 0]

In [145]:
dangerous_cluster

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
0,1,17.4242,-61.6573,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304,dlattach;topic=41020;attach=7045;image,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304,dlattach;topic=41020;attach=7045;image,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304,dlattach;topic=41020;attach=7045;image,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304,dlattach;topic=41020;attach=7045;image,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304,dlattach;topic=41020;attach=7045;image
1,1,32.091,-44.243,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200,MPWM8At5,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200,MPWM8At5,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200,MPWM8At5,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200,MPWM8At5,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200,MPWM8At5
2,1,-34.3421,32.7929,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200,/imgbasa/17896173683jpg,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200,/imgbasa/17896173683jpg,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200,/imgbasa/17896173683jpg,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200,/imgbasa/17896173683jpg,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200,/imgbasa/17896173683jpg
6,1,-33.3099,46.2332,13,13,f00d514a360a9926f1a54f99a9e7a942,REQUEST_PATH,REQUEST_PATH,200,/mail/spb/vkvasniknsf/Inbox/B481571280728846C1...,13,13,f00d514a360a9926f1a54f99a9e7a942,REQUEST_PATH,REQUEST_PATH,200,/mail/spb/vkvasniknsf/Inbox/B481571280728846C1...,13,13,f00d514a360a9926f1a54f99a9e7a942,REQUEST_PATH,REQUEST_PATH,200,/mail/spb/vkvasniknsf/Inbox/B481571280728846C1...,13,13,f00d514a360a9926f1a54f99a9e7a942,REQUEST_PATH,REQUEST_PATH,200,/mail/spb/vkvasniknsf/Inbox/B481571280728846C1...,13,13,f00d514a360a9926f1a54f99a9e7a942,REQUEST_PATH,REQUEST_PATH,200,/mail/spb/vkvasniknsf/Inbox/B481571280728846C1...
7,1,-25.1058,-3.92882,15,15,b7caea0cc6d15508ff30f95f3e599160,RESPONSE_HEADERS,RESPONSE_HEADERS.Age,200,1443,15,15,b7caea0cc6d15508ff30f95f3e599160,RESPONSE_HEADERS,RESPONSE_HEADERS.Age,200,1443,15,15,b7caea0cc6d15508ff30f95f3e599160,RESPONSE_HEADERS,RESPONSE_HEADERS.Age,200,1443,15,15,b7caea0cc6d15508ff30f95f3e599160,RESPONSE_HEADERS,RESPONSE_HEADERS.Age,200,1443,15,15,b7caea0cc6d15508ff30f95f3e599160,RESPONSE_HEADERS,RESPONSE_HEADERS.Age,200,1443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5514,1,-48.1102,59.6277,997,990,c07dc8984b8def157ef49ee21f31fde5,REQUEST_XML,REQUEST_XML./s:Envelope/s:Body/DeviceMetadataB...,302,DOID:HIDVID2717PIDF006REV0318,997,990,c07dc8984b8def157ef49ee21f31fde5,REQUEST_XML,REQUEST_XML./s:Envelope/s:Body/DeviceMetadataB...,302,DOID:HIDVID2717PIDF006REV0318,997,990,c07dc8984b8def157ef49ee21f31fde5,REQUEST_XML,REQUEST_XML./s:Envelope/s:Body/DeviceMetadataB...,302,DOID:HIDVID2717PIDF006REV0318,997,990,c07dc8984b8def157ef49ee21f31fde5,REQUEST_XML,REQUEST_XML./s:Envelope/s:Body/DeviceMetadataB...,302,DOID:HIDVID2717PIDF006REV0318,997,990,c07dc8984b8def157ef49ee21f31fde5,REQUEST_XML,REQUEST_XML./s:Envelope/s:Body/DeviceMetadataB...,302,DOID:HIDVID2717PIDF006REV0318
5516,1,-31.1713,14.2039,1000,993,2f0f2a89b9742042d11bee029fdb9d3b,REQUEST_HEADERS,REQUEST_HEADERS.X-Forwarded-For,200,91185899,1000,993,2f0f2a89b9742042d11bee029fdb9d3b,REQUEST_HEADERS,REQUEST_HEADERS.X-Forwarded-For,200,91185899,1000,993,2f0f2a89b9742042d11bee029fdb9d3b,REQUEST_HEADERS,REQUEST_HEADERS.X-Forwarded-For,200,91185899,1000,993,2f0f2a89b9742042d11bee029fdb9d3b,REQUEST_HEADERS,

In [146]:
safe_cluster

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
3,0,0.994176,31.1139,7,7,950ebbb818650193f9328d75453e69cb,REQUEST_HEADERS,Referer,200,file:///Applications/Install/639E2536C2764671B...,7,7,950ebbb818650193f9328d75453e69cb,REQUEST_HEADERS,Referer,200,file:///Applications/Install/639E2536C2764671B...,7,7,950ebbb818650193f9328d75453e69cb,REQUEST_HEADERS,Referer,200,file:///Applications/Install/639E2536C2764671B...,7,7,950ebbb818650193f9328d75453e69cb,REQUEST_HEADERS,Referer,200,file:///Applications/Install/639E2536C2764671B...,7,7,950ebbb818650193f9328d75453e69cb,REQUEST_HEADERS,Referer,200,file:///Applications/Install/639E2536C2764671B...
4,0,1.00028,31.1198,9,9,d2734f4ab17d4c4cd638f52a8628402a,REQUEST_HEADERS,Referer,200,file:///Applications/Install/65621A5D0DDE45B1A...,9,9,d2734f4ab17d4c4cd638f52a8628402a,REQUEST_HEADERS,Referer,200,file:///Applications/Install/65621A5D0DDE45B1A...,9,9,d2734f4ab17d4c4cd638f52a8628402a,REQUEST_HEADERS,Referer,200,file:///Applications/Install/65621A5D0DDE45B1A...,9,9,d2734f4ab17d4c4cd638f52a8628402a,REQUEST_HEADERS,Referer,200,file:///Applications/Install/65621A5D0DDE45B1A...,9,9,d2734f4ab17d4c4cd638f52a8628402a,REQUEST_HEADERS,Referer,200,file:///Applications/Install/65621A5D0DDE45B1A...
5,0,-3.55185,47.1862,11,11,a82230c545dcbc51e830255835ae25d6,REQUEST_ARGS,file,200,///images/stories/1/t1jpg,11,11,a82230c545dcbc51e830255835ae25d6,REQUEST_ARGS,file,200,///images/stories/1/t1jpg,11,11,a82230c545dcbc51e830255835ae25d6,REQUEST_ARGS,file,200,///images/stories/1/t1jpg,11,11,a82230c545dcbc51e830255835ae25d6,REQUEST_ARGS,file,200,///images/stories/1/t1jpg,11,11,a82230c545dcbc51e830255835ae25d6,REQUEST_ARGS,file,200,///images/stories/1/t1jpg
9,0,39.1341,-18.8973,23,23,ca33d909a87e1d1eea8791866ed106fb,REQUEST_COOKIES,anj,200,dTM7kM4/C/9P:57Xwb7NPL6xPSAeE=hrgJ7vjwyY2INN9:...,23,23,ca33d909a87e1d1eea8791866ed106fb,REQUEST_COOKIES,anj,200,dTM7kM4/C/9P:57Xwb7NPL6xPSAeE=hrgJ7vjwyY2INN9:...,23,23,ca33d909a87e1d1eea8791866ed106fb,REQUEST_COOKIES,anj,200,dTM7kM4/C/9P:57Xwb7NPL6xPSAeE=hrgJ7vjwyY2INN9:...,23,23,ca33d909a87e1d1eea8791866ed106fb,REQUEST_COOKIES,anj,200,dTM7kM4/C/9P:57Xwb7NPL6xPSAeE=hrgJ7vjwyY2INN9:...,23,23,ca33d909a87e1d1eea8791866ed106fb,REQUEST_COOKIES,anj,200,dTM7kM4/C/9P:57Xwb7NPL6xPSAeE=hrgJ7vjwyY2INN9:...
10,0,6.49386,60.2941,24,24,0bce3849ead8521394088d6599bf468b,REQUEST_ARGS,pll_language,200,file:///windows/winini,24,24,0bce3849ead8521394088d6599bf468b,REQUEST_ARGS,pll_language,200,file:///windows/winini,24,24,0bce3849ead8521394088d6599bf468b,REQUEST_ARGS,pll_language,200,file:///windows/winini,24,24,0bce3849ead8521394088d6599bf468b,REQUEST_ARGS,pll_language,200,file:///windows/winini,24,24,0bce3849ead8521394088d6599bf468b,REQUEST_ARGS,pll_language,200,file:///windows/winini
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5500,0,16.2687,31.8272,974,967,ad65b3b71aa832e10fcc2aad2e18ad1b,REQUEST_PATH,REQUEST_PATH,200,/documents/nationalplan,974,967,ad65b3b71aa832e10fcc2aad2e18ad1b,REQUEST_PATH,REQUEST_PATH,200,/documents/nationalplan,974,967,ad65b3b71aa832e10fcc2aad2e18ad1b,REQUEST_PATH,REQUEST_PATH,200,/documents/nationalplan,974,967,ad65b3b71aa832e10fcc2aad2e18ad1b,REQUEST_PATH,REQUEST_PATH,200,/documents/nationalplan,974,967,ad65b3b71aa832e10fcc2aad2e18ad1b,REQUEST_PATH,REQUEST_PATH,200,/documents/nationalplan
5505,0,35.8608,-20.2857,981,974,1ba07ba33d164789e5547683be31a44d,REQUEST_COOKIES,anj,200,dTM7kM4/KUy5eyf/Eo/KEzIaUcuNQki0uf5Ybh23TSiFUX...,981,974,1ba07ba33d164789e5547683be31a44d,REQUEST_COOKIES,anj,200,dTM7kM4/KUy5eyf/Eo/KEzIaUcuNQki0uf5Ybh23TSiFUX...,981,974,1ba07ba33d164789e5547683be31a44d,REQUEST_COOKIES,anj,200,dTM7kM4/KUy5eyf/Eo/KEzIaUcuNQki0uf5Ybh23TSiFUX...,981,974,1ba07ba33d164789e5547683be31a44d,REQUEST_COOKIES,anj,200,dTM7kM4/KUy5eyf/Eo/KEzIaUcuNQki0uf5Ybh23TSiFUX...,981,974,1ba07ba33d164789e5547683be31a44d,RE

In [129]:
#Кластеризация потенциально опасного кластера с помощью DBSCAN
samples =dangerous_cluster[9].tolist()
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3))
vectors = vectorizer.fit_transform(samples)


res = DBSCAN(eps=0.3, min_samples=10).fit_predict(vectors)
new_df1 = pd.DataFrame(data=res, columns=['label'])
set(res)

{-1, 0, 1, 2, 3, 4, 5}

Получили 7 разных кластеров

In [130]:
#Визуализация
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(vectors)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [131]:
new_df1["x_component"]=X_embedded[:,0]
new_df1["y_component"]=X_embedded[:,1]

In [132]:
import plotly.express as px
    
fig = px.scatter(new_df1, x="x_component", y="y_component", color = "label", size_max=60)
fig.update_layout(
     height=800)
fig.show()

In [134]:
foo = new_df1.to_numpy()
bar = dangerous_cluster.to_numpy()
new_df1 = np.concatenate((foo, bar), axis=1)
new_df1 = pd.DataFrame(new_df1)

noise1_cluster = new_df1.loc[new_df1[0] == -1]
second_cluster = new_df1.loc[new_df1[0] == 0]
third_cluster = new_df1.loc[new_df1[0] == 1]
fourth_cluster = new_df1.loc[new_df1[0] == 2]
fifth_cluster = new_df1.loc[new_df1[0] == 3]
sixth_cluster = new_df1.loc[new_df1[0] == 4]
seventh_cluster = new_df1.loc[new_df1[0] == 5]



In [135]:
noise1_cluster

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
0,-1,-58.2376,15.3547,1,17.4242,-61.6573,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304,dlattach;topic=41020;attach=7045;image,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304,dlattach;topic=41020;attach=7045;image,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304,dlattach;topic=41020;attach=7045;image,1,17.4242,-61.6573,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304,dlattach;topic=41020;attach=7045;image,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304,dlattach;topic=41020;attach=7045;image,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304,dlattach;topic=41020;attach=7045;image
1,-1,-21.4606,-1.76326,1,32.091,-44.243,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200,MPWM8At5,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200,MPWM8At5,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200,MPWM8At5,1,32.091,-44.243,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200,MPWM8At5,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200,MPWM8At5,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200,MPWM8At5
2,-1,-45.7235,-27.3598,1,-34.3421,32.7929,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200,/imgbasa/17896173683jpg,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200,/imgbasa/17896173683jpg,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200,/imgbasa/17896173683jpg,1,-34.3421,32.7929,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200,/imgbasa/17896173683jpg,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200,/imgbasa/17896173683jpg,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200,/imgbasa/17896173683jpg
3,-1,-37.0178,-11.5716,1,-33.3099,46.2332,13,13,f00d514a360a9926f1a54f99a9e7a942,REQUEST_PATH,REQUEST_PATH,200,/mail/spb/vkvasniknsf/Inbox/B481571280728846C1...,13,13,f00d514a360a9926f1a54f99a9e7a942,REQUEST_PATH,REQUEST_PATH,200,/mail/spb/vkvasniknsf/Inbox/B481571280728846C1...,13,13,f00d514a360a9926f1a54f99a9e7a942,REQUEST_PATH,REQUEST_PATH,200,/mail/spb/vkvasniknsf/Inbox/B481571280728846C1...,1,-33.3099,46.2332,13,13,f00d514a360a9926f1a54f99a9e7a942,REQUEST_PATH,REQUEST_PATH,200,/mail/spb/vkvasniknsf/Inbox/B481571280728846C1...,13,13,f00d514a360a9926f1a54f99a9e7a942,REQUEST_PATH,REQUEST_PATH,200,/mail/spb/vkvasniknsf/Inbox/B481571280728846C1...,13,13,f00d514a360a9926f1a54f99a9e7a942,REQUEST_PATH,REQUEST_PATH,200,/mail/spb/vkvasniknsf/Inbox/B481571280728846C1...
4,-1,-13.0446,-33.0856,1,-25.1058,-3.92882,15,15,b7caea0cc6d15508ff30f95f3e599160,RESPONSE_HEADERS,RESPONSE_HEADERS.Age,200,1443,15,15,b7caea0cc6d15508ff30f95f3e599160,RESPONSE_HEADERS,RESPONSE_HEADERS.Age,200,1443,15,15,b7caea0cc6d15508ff30f95f3e599160,RESPONSE_HEADERS,RESPONSE_HEADERS.Age,200,1443,1,-25.1058,-3.92882,15,15,b7caea0cc6d15508ff30f95f3e599160,RESPONSE_HEADERS,RESPONSE_HEADERS.Age,200,1443,15,15,b7caea0cc6d15508ff30f95f3e599160,RESPONSE_HEADERS,RESPONSE_HEADERS.Age,200,1443,15,15,b7caea0cc6d15508ff30f95f3e599160,RESPONSE_HEADERS,RESPONSE_HEADERS.Age,200,1443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3302,-1,47.6572,27.1819,1,-48.1102,59.6277,997,990,c07dc8984b8def157ef49ee21f31fde5,REQUEST_XML,REQUEST_XML./s:Envelope/s:Body/DeviceMetadataB...,302,DOID:HIDVID2717PIDF006REV0318,997,990,c07dc8984b8def157ef49ee21f31fde5,REQUEST_XML,REQUEST_XML./s:Envelope/s:Body/DeviceMetadataB...,302,DOID:HIDVID2717PI

In [136]:
second_cluster

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
6,0,7.22864,78.7146,1,14.6146,-90.3462,26,26,56616167ae6cff7829b5d8a2679e7af9,REQUEST_ARGS,id,200,11printchr492bchr552bchr732bchr532bchr512bchr4...,26,26,56616167ae6cff7829b5d8a2679e7af9,REQUEST_ARGS,id,200,11printchr492bchr552bchr732bchr532bchr512bchr4...,26,26,56616167ae6cff7829b5d8a2679e7af9,REQUEST_ARGS,id,200,11printchr492bchr552bchr732bchr532bchr512bchr4...,1,14.6146,-90.3462,26,26,56616167ae6cff7829b5d8a2679e7af9,REQUEST_ARGS,id,200,11printchr492bchr552bchr732bchr532bchr512bchr4...,26,26,56616167ae6cff7829b5d8a2679e7af9,REQUEST_ARGS,id,200,11printchr492bchr552bchr732bchr532bchr512bchr4...,26,26,56616167ae6cff7829b5d8a2679e7af9,REQUEST_ARGS,id,200,11printchr492bchr552bchr732bchr532bchr512bchr4...
10,0,8.2071,78.2353,1,14.0994,-90.3322,35,35,74061910e04d75450cd6fc5a41f9380f,REQUEST_ARGS,products_id,200,65printchr492bchr552bchr732bchr532bchr512bchr4...,35,35,74061910e04d75450cd6fc5a41f9380f,REQUEST_ARGS,products_id,200,65printchr492bchr552bchr732bchr532bchr512bchr4...,35,35,74061910e04d75450cd6fc5a41f9380f,REQUEST_ARGS,products_id,200,65printchr492bchr552bchr732bchr532bchr512bchr4...,1,14.0994,-90.3322,35,35,74061910e04d75450cd6fc5a41f9380f,REQUEST_ARGS,products_id,200,65printchr492bchr552bchr732bchr532bchr512bchr4...,35,35,74061910e04d75450cd6fc5a41f9380f,REQUEST_ARGS,products_id,200,65printchr492bchr552bchr732bchr532bchr512bchr4...,35,35,74061910e04d75450cd6fc5a41f9380f,REQUEST_ARGS,products_id,200,65printchr492bchr552bchr732bchr532bchr512bchr4...
109,0,7.35214,78.482,1,14.2735,-89.6584,373,370,f0c5c7d9485f27172c66ce2ba8539148,REQUEST_ARGS,pt_cat,200,8printchr492bchr552bchr732bchr532bchr512bchr48...,373,370,f0c5c7d9485f27172c66ce2ba8539148,REQUEST_ARGS,pt_cat,200,8printchr492bchr552bchr732bchr532bchr512bchr48...,373,370,f0c5c7d9485f27172c66ce2ba8539148,REQUEST_ARGS,pt_cat,200,8printchr492bchr552bchr732bchr532bchr512bchr48...,1,14.2735,-89.6584,373,370,f0c5c7d9485f27172c66ce2ba8539148,REQUEST_ARGS,pt_cat,200,8printchr492bchr552bchr732bchr532bchr512bchr48...,373,370,f0c5c7d9485f27172c66ce2ba8539148,REQUEST_ARGS,pt_cat,200,8printchr492bchr552bchr732bchr532bchr512bchr48...,373,370,f0c5c7d9485f27172c66ce2ba8539148,REQUEST_ARGS,pt_cat,200,8printchr492bchr552bchr732bchr532bchr512bchr48...
185,0,8.19734,78.1949,1,14.1504,-90.4286,619,616,ef41c8895d75bbfa60db1b32dac0ac8b,REQUEST_ARGS,ContentPage,301,55printchr492bchr552bchr732bchr532bchr512bchr4...,619,616,ef41c8895d75bbfa60db1b32dac0ac8b,REQUEST_ARGS,ContentPage,301,55printchr492bchr552bchr732bchr532bchr512bchr4...,619,616,ef41c8895d75bbfa60db1b32dac0ac8b,REQUEST_ARGS,ContentPage,301,55printchr492bchr552bchr732bchr532bchr512bchr4...,1,14.1504,-90.4286,619,616,ef41c8895d75bbfa60db1b32dac0ac8b,REQUEST_ARGS,ContentPage,301,55printchr492bchr552bchr732bchr532bchr512bchr4...,619,616,ef41c8895d75bbfa60db1b32dac0ac8b,REQUEST_ARGS,ContentPage,301,55printchr492bchr552bchr732bchr532bchr512bchr4...,619,616,ef41c8895d75bbfa60db1b32dac0ac8b,REQUEST_ARGS,ContentPage,301,55printchr492bchr552bchr732bchr532bchr512bchr4...
688,0,7.92894,78.8794,1,14.6777,-90.2348,245,245,9eae1f8ea9d3431ab29bc2a467b74b2a,REQUEST_ARGS,pt_id,200,7printchr492bchr552bchr732bchr532bchr512bchr48...,245,245,9eae1f8ea9d3431ab29bc2a467b74b2a,REQUEST_ARGS,pt_id,200,7printchr492bchr552bchr732bchr532bchr512bchr48...,245,245,9eae1f8ea9d3431ab29bc2a467b74b2a,REQUEST_ARGS,pt_id,200,7printchr492bchr552bchr732bchr532bchr512bchr48...,1,14.6777,-90.2348,245,245,9eae1f8ea9d3431ab29bc2a467b74b2a,REQUEST_ARGS,pt_id,200,7printchr492bchr552bchr732bchr532bchr512bchr48...,245,245,9eae1f8ea9d3431ab29bc2a467b74b2a,REQUEST_ARGS,pt_id,200,7printchr492bchr552bchr732bchr532bchr512bchr48...,245,245,9eae1f8ea9d3431ab29bc2a467b74b2a,REQUEST_ARGS,pt_id,200,7printchr492bchr552bchr732bchr532bchr512bchr48...
725,0,6.80495,79.0382,1,14.9029,-89.8051,376,376,c21044971b0fd0f23b4a2b770749f572,REQUEST_ARGS,id,200,223

In [137]:
third_cluster

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
11,1,22.0342,57.4445,1,0.233206,-78.8799,40,40,dbc09a238a5d6065f7dd6b94ec970f18,REQUEST_ARGS,gid,200,907printchr49chr55chr73chr53chr51chr48chr86chr...,40,40,dbc09a238a5d6065f7dd6b94ec970f18,REQUEST_ARGS,gid,200,907printchr49chr55chr73chr53chr51chr48chr86chr...,40,40,dbc09a238a5d6065f7dd6b94ec970f18,REQUEST_ARGS,gid,200,907printchr49chr55chr73chr53chr51chr48chr86chr...,1,0.233206,-78.8799,40,40,dbc09a238a5d6065f7dd6b94ec970f18,REQUEST_ARGS,gid,200,907printchr49chr55chr73chr53chr51chr48chr86chr...,40,40,dbc09a238a5d6065f7dd6b94ec970f18,REQUEST_ARGS,gid,200,907printchr49chr55chr73chr53chr51chr48chr86chr...,40,40,dbc09a238a5d6065f7dd6b94ec970f18,REQUEST_ARGS,gid,200,907printchr49chr55chr73chr53chr51chr48chr86chr...
53,1,28.0609,56.0852,1,3.98432,-76.4256,162,162,2aec1aa2620598c32be025e52e4fe09a,REQUEST_ARGS,tag,200,'printchr49chr55chr73chr53chr51chr48chr86chr65...,162,162,2aec1aa2620598c32be025e52e4fe09a,REQUEST_ARGS,tag,200,'printchr49chr55chr73chr53chr51chr48chr86chr65...,162,162,2aec1aa2620598c32be025e52e4fe09a,REQUEST_ARGS,tag,200,'printchr49chr55chr73chr53chr51chr48chr86chr65...,1,3.98432,-76.4256,162,162,2aec1aa2620598c32be025e52e4fe09a,REQUEST_ARGS,tag,200,'printchr49chr55chr73chr53chr51chr48chr86chr65...,162,162,2aec1aa2620598c32be025e52e4fe09a,REQUEST_ARGS,tag,200,'printchr49chr55chr73chr53chr51chr48chr86chr65...,162,162,2aec1aa2620598c32be025e52e4fe09a,REQUEST_ARGS,tag,200,'printchr49chr55chr73chr53chr51chr48chr86chr65...
66,1,23.046,52.3816,1,-2.80316,-75.8137,210,209,5b161e7a44651524652cee977f969a87,REQUEST_ARGS,gid,200,769printchr49chr55chr73chr53chr51chr48chr86chr...,210,209,5b161e7a44651524652cee977f969a87,REQUEST_ARGS,gid,200,769printchr49chr55chr73chr53chr51chr48chr86chr...,210,209,5b161e7a44651524652cee977f969a87,REQUEST_ARGS,gid,200,769printchr49chr55chr73chr53chr51chr48chr86chr...,1,-2.80316,-75.8137,210,209,5b161e7a44651524652cee977f969a87,REQUEST_ARGS,gid,200,769printchr49chr55chr73chr53chr51chr48chr86chr...,210,209,5b161e7a44651524652cee977f969a87,REQUEST_ARGS,gid,200,769printchr49chr55chr73chr53chr51chr48chr86chr...,210,209,5b161e7a44651524652cee977f969a87,REQUEST_ARGS,gid,200,769printchr49chr55chr73chr53chr51chr48chr86chr...
76,1,20.7404,54.8933,1,-1.68392,-77.9607,239,238,5b397741f04d9ae3a50e1d73342a0ad6,REQUEST_ARGS,do,200,catprintchr49chr55chr73chr53chr51chr48chr86chr...,239,238,5b397741f04d9ae3a50e1d73342a0ad6,REQUEST_ARGS,do,200,catprintchr49chr55chr73chr53chr51chr48chr86chr...,239,238,5b397741f04d9ae3a50e1d73342a0ad6,REQUEST_ARGS,do,200,catprintchr49chr55chr73chr53chr51chr48chr86chr...,1,-1.68392,-77.9607,239,238,5b397741f04d9ae3a50e1d73342a0ad6,REQUEST_ARGS,do,200,catprintchr49chr55chr73chr53chr51chr48chr86chr...,239,238,5b397741f04d9ae3a50e1d73342a0ad6,REQUEST_ARGS,do,200,catprintchr49chr55chr73chr53chr51chr48chr86chr...,239,238,5b397741f04d9ae3a50e1d73342a0ad6,REQUEST_ARGS,do,200,catprintchr49chr55chr73chr53chr51chr48chr86chr...
81,1,21.2013,57.152,1,-0.290019,-79.3057,261,260,49a275bac9fe7cf15c242d7ae6e4a4e8,REQUEST_ARGS,ID,200,1147printchr49chr55chr73chr53chr51chr48chr86ch...,261,260,49a275bac9fe7cf15c242d7ae6e4a4e8,REQUEST_ARGS,ID,200,1147printchr49chr55chr73chr53chr51chr48chr86ch...,261,260,49a275bac9fe7cf15c242d7ae6e4a4e8,REQUEST_ARGS,ID,200,1147printchr49chr55chr73chr53chr51chr48chr86ch...,1,-0.290019,-79.3057,261,260,49a275bac9fe7cf15c242d7ae6e4a4e8,REQUEST_ARGS,ID,200,1147printchr49chr55chr73chr53chr51chr48chr86ch...,261,260,49a275bac9fe7cf15c242d7ae6e4a4e8,REQUEST_ARGS,ID,200,1147printchr49chr55chr73chr53chr51chr48chr86ch...,261,260,49a275bac9fe7cf15c242d7ae6e4a4e8,REQUEST_ARGS,ID,200,1147printchr49chr55chr73chr53chr51chr48chr86ch...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3206,1,21

In [138]:
fourth_cluster

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
14,2,-15.8262,20.8278,1,8.77534,69.5968,44,44,ce1e18f9e42cb001f70c50663461c352,REQUEST_ARGS_KEYS,a?a?aa??a?aa?aa??????a?a??a?a?a?aa??a?aa?aa???...,200,aaaaaaaaaaaaaaaaaaaaaaaaaaaaa,44,44,ce1e18f9e42cb001f70c50663461c352,REQUEST_ARGS_KEYS,a?a?aa??a?aa?aa??????a?a??a?a?a?aa??a?aa?aa???...,200,aaaaaaaaaaaaaaaaaaaaaaaaaaaaa,44,44,ce1e18f9e42cb001f70c50663461c352,REQUEST_ARGS_KEYS,a?a?aa??a?aa?aa??????a?a??a?a?a?aa??a?aa?aa???...,200,aaaaaaaaaaaaaaaaaaaaaaaaaaaaa,1,8.77534,69.5968,44,44,ce1e18f9e42cb001f70c50663461c352,REQUEST_ARGS_KEYS,a?a?aa??a?aa?aa??????a?a??a?a?a?aa??a?aa?aa???...,200,aaaaaaaaaaaaaaaaaaaaaaaaaaaaa,44,44,ce1e18f9e42cb001f70c50663461c352,REQUEST_ARGS_KEYS,a?a?aa??a?aa?aa??????a?a??a?a?a?aa??a?aa?aa???...,200,aaaaaaaaaaaaaaaaaaaaaaaaaaaaa,44,44,ce1e18f9e42cb001f70c50663461c352,REQUEST_ARGS_KEYS,a?a?aa??a?aa?aa??????a?a??a?a?a?aa??a?aa?aa???...,200,aaaaaaaaaaaaaaaaaaaaaaaaaaaaa
25,2,-14.8869,20.1347,1,9.47556,69.5814,86,86,4026304aa39ae07afc7c281f1f5ad6f8,REQUEST_ARGS_KEYS,Aï¿½ï¿½A?A?A?Aï¿½A?Aï¿½ï¿½A%C,200,AAAAAAAAC,86,86,4026304aa39ae07afc7c281f1f5ad6f8,REQUEST_ARGS_KEYS,Aï¿½ï¿½A?A?A?Aï¿½A?Aï¿½ï¿½A%C,200,AAAAAAAAC,86,86,4026304aa39ae07afc7c281f1f5ad6f8,REQUEST_ARGS_KEYS,Aï¿½ï¿½A?A?A?Aï¿½A?Aï¿½ï¿½A%C,200,AAAAAAAAC,1,9.47556,69.5814,86,86,4026304aa39ae07afc7c281f1f5ad6f8,REQUEST_ARGS_KEYS,Aï¿½ï¿½A?A?A?Aï¿½A?Aï¿½ï¿½A%C,200,AAAAAAAAC,86,86,4026304aa39ae07afc7c281f1f5ad6f8,REQUEST_ARGS_KEYS,Aï¿½ï¿½A?A?A?Aï¿½A?Aï¿½ï¿½A%C,200,AAAAAAAAC,86,86,4026304aa39ae07afc7c281f1f5ad6f8,REQUEST_ARGS_KEYS,Aï¿½ï¿½A?A?A?Aï¿½A?Aï¿½ï¿½A%C,200,AAAAAAAAC
194,2,-15.4419,21.2412,1,8.82812,69.4688,652,649,f2a3ed6273fa3908274bcc4d3d5faa11,REQUEST_ARGS_KEYS,a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?aÃ...,200,aaaaaaaaaaaaaaaaaaaaaaa,652,649,f2a3ed6273fa3908274bcc4d3d5faa11,REQUEST_ARGS_KEYS,a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?aÃ...,200,aaaaaaaaaaaaaaaaaaaaaaa,652,649,f2a3ed6273fa3908274bcc4d3d5faa11,REQUEST_ARGS_KEYS,a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?aÃ...,200,aaaaaaaaaaaaaaaaaaaaaaa,1,8.82812,69.4688,652,649,f2a3ed6273fa3908274bcc4d3d5faa11,REQUEST_ARGS_KEYS,a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?aÃ...,200,aaaaaaaaaaaaaaaaaaaaaaa,652,649,f2a3ed6273fa3908274bcc4d3d5faa11,REQUEST_ARGS_KEYS,a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?aÃ...,200,aaaaaaaaaaaaaaaaaaaaaaa,652,649,f2a3ed6273fa3908274bcc4d3d5faa11,REQUEST_ARGS_KEYS,a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?a?aÃ...,200,aaaaaaaaaaaaaaaaaaaaaaa
497,2,-14.9704,20.5916,1,9.22173,69.5074,661,660,a10cd92cff50a2d673660307c2e35fc2,REQUEST_ARGS_KEYS,a?a?a?a?a?a?a?a?a?a?aÃ?ÃÂ¯Ã?%,200,aaaaaaaaaaa,661,660,a10cd92cff50a2d673660307c2e35fc2,REQUEST_ARGS_KEYS,a?a?a?a?a?a?a?a?a?a?aÃ?ÃÂ¯Ã?%,200,aaaaaaaaaaa,661,660,a10cd92cff50a2d673660307c2e35fc2,REQUEST_ARGS_KEYS,a?a?a?a?a?a?a?a?a?a?aÃ?ÃÂ¯Ã?%,200,aaaaaaaaaaa,1,9.22173,69.5074,661,660,a10cd92cff50a2d673660307c2e35fc2,REQUEST_ARGS_KEYS,a?a?a?a?a?a?a?a?a?a?aÃ?ÃÂ¯Ã?%,200,aaaaaaaaaaa,661,660,a10cd92cff50a2d673660307c2e35fc2,REQUEST_ARGS_KEYS,a?a?a?a?a?a?a?a?a?a?aÃ?ÃÂ¯Ã?%,200,aaaaaaaaaaa,661,660,a10cd92cff50a2d673660307c2e35fc2,REQUEST_ARGS_KEYS,a?a?a?a?a?a?a?a?a?a?aÃ?ÃÂ¯Ã?%,200,aaaaaaaaaaa
520,2,-15.8622,20.8728,1,8.76951,69.609,727,726,538cfea775abef64bea949da3957cf79,REQUEST_ARGS_KEYS,a?a?aa??a?aA????A???A??A?A??a?a??a?a?a?aa??a?a...,200,aaaaaaAAAAAaaaaaaaaaAAAAAaaaaaAA,727,726,538cfea775abef64bea949da3957cf79,REQUEST_ARGS_KEYS,a?a?aa??a?aA????A???A??A?A??a?a??a?a?a?aa??a?a...,200,aaaaaaAAAAAaaaaaaaaaAAAAAaaaaaAA,727,726,538cfea775abef64bea949da3957cf79,REQUEST_ARGS_KEYS,a?a?aa??a?aA????A???A??A?A??a?a??a?a?a?aa??a?a...,200,aaaaaaAAAAAaaaaaaaaaAAAAAaaaaaAA,1,8.76951,69.609,727,726,538cfea775abef64bea949da3957cf79,REQUEST_ARGS_KEYS,a?a?aa??a?aA????A???A??A?A??a?a??a?a?a?aa??a?a...,200,aaaaaaAAAAAaaaaaaaaaAAAAAaaaaaAA,727,726,538cfea775abef64bea949da3957cf79,REQUEST_ARGS_KEYS,

In [139]:
fifth_cluster

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
41,3,-27.8065,-54.1334,1,-66.8446,6.53596,120,120,6107702942e26706af81ae2381804221,REQUEST_ARGS_KEYS,%25252525252525252525252526page%25252525252525...,200,25252525252525252525252526page2525252525252525...,120,120,6107702942e26706af81ae2381804221,REQUEST_ARGS_KEYS,%25252525252525252525252526page%25252525252525...,200,25252525252525252525252526page2525252525252525...,120,120,6107702942e26706af81ae2381804221,REQUEST_ARGS_KEYS,%25252525252525252525252526page%25252525252525...,200,25252525252525252525252526page2525252525252525...,1,-66.8446,6.53596,120,120,6107702942e26706af81ae2381804221,REQUEST_ARGS_KEYS,%25252525252525252525252526page%25252525252525...,200,25252525252525252525252526page2525252525252525...,120,120,6107702942e26706af81ae2381804221,REQUEST_ARGS_KEYS,%25252525252525252525252526page%25252525252525...,200,25252525252525252525252526page2525252525252525...,120,120,6107702942e26706af81ae2381804221,REQUEST_ARGS_KEYS,%25252525252525252525252526page%25252525252525...,200,25252525252525252525252526page2525252525252525...
48,3,-27.4523,-53.8754,1,-66.5422,6.49626,147,147,fb4e3e5fc036edd7a0650886cbdb2bd5,REQUEST_ARGS_KEYS,%25252525252525252525252526page%25252525252525...,200,25252525252525252525252526page2525252525252525...,147,147,fb4e3e5fc036edd7a0650886cbdb2bd5,REQUEST_ARGS_KEYS,%25252525252525252525252526page%25252525252525...,200,25252525252525252525252526page2525252525252525...,147,147,fb4e3e5fc036edd7a0650886cbdb2bd5,REQUEST_ARGS_KEYS,%25252525252525252525252526page%25252525252525...,200,25252525252525252525252526page2525252525252525...,1,-66.5422,6.49626,147,147,fb4e3e5fc036edd7a0650886cbdb2bd5,REQUEST_ARGS_KEYS,%25252525252525252525252526page%25252525252525...,200,25252525252525252525252526page2525252525252525...,147,147,fb4e3e5fc036edd7a0650886cbdb2bd5,REQUEST_ARGS_KEYS,%25252525252525252525252526page%25252525252525...,200,25252525252525252525252526page2525252525252525...,147,147,fb4e3e5fc036edd7a0650886cbdb2bd5,REQUEST_ARGS_KEYS,%25252525252525252525252526page%25252525252525...,200,25252525252525252525252526page2525252525252525...
108,3,-24.6892,-55.6018,1,-65.898,9.02694,371,368,c664bf968cbebdd4c02cf43298721041,REQUEST_ARGS_KEYS,/tags/forums/%2525252525D0%2525252525B1%252525...,200,/tags/forums/2525252525D02525252525B1252525252...,371,368,c664bf968cbebdd4c02cf43298721041,REQUEST_ARGS_KEYS,/tags/forums/%2525252525D0%2525252525B1%252525...,200,/tags/forums/2525252525D02525252525B1252525252...,371,368,c664bf968cbebdd4c02cf43298721041,REQUEST_ARGS_KEYS,/tags/forums/%2525252525D0%2525252525B1%252525...,200,/tags/forums/2525252525D02525252525B1252525252...,1,-65.898,9.02694,371,368,c664bf968cbebdd4c02cf43298721041,REQUEST_ARGS_KEYS,/tags/forums/%2525252525D0%2525252525B1%252525...,200,/tags/forums/2525252525D02525252525B1252525252...,371,368,c664bf968cbebdd4c02cf43298721041,REQUEST_ARGS_KEYS,/tags/forums/%2525252525D0%2525252525B1%252525...,200,/tags/forums/2525252525D02525252525B1252525252...,371,368,c664bf968cbebdd4c02cf43298721041,REQUEST_ARGS_KEYS,/tags/forums/%2525252525D0%2525252525B1%252525...,200,/tags/forums/2525252525D02525252525B1252525252...
118,3,-32.3021,-47.9697,1,-65.8337,0.764867,414,411,45f89b82184fc08200e4410b5230ef50,REQUEST_ARGS_KEYS,%252525252526page%25252525253D3%252525252526pa...,200,252525252526page25252525253D3252525252526page2...,414,411,45f89b82184fc08200e4410b5230ef50,REQUEST_ARGS_KEYS,%252525252526page%25252525253D3%252525252526pa...,200,252525252526page25252525253D3252525252526page2...,414,411,45f89b82184fc08200e4410b5230ef50,REQUEST_ARGS_KEYS,%252525252526page%25252525253D3%252525252526pa...,200,252525252526page25252525253D3252525252526page2...,1,-65.8337,0.764867,414,411,45f89b82184fc08200e4410b5230ef50,REQUEST_ARGS_KEYS,%252525252526page%25252525253D3%252525252526pa...,200,252525252526page25252525253D3252525252526page2...,414,411,45f89b82184fc082

In [140]:
sixth_cluster

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
117,4,-50.9228,-18.2276,1,24.5046,77.1921,413,410,8710d0452ec15257b16b9e690d945989,REQUEST_GET_ARGS,cat,200,28/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,413,410,8710d0452ec15257b16b9e690d945989,REQUEST_GET_ARGS,cat,200,28/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,413,410,8710d0452ec15257b16b9e690d945989,REQUEST_GET_ARGS,cat,200,28/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,1,24.5046,77.1921,413,410,8710d0452ec15257b16b9e690d945989,REQUEST_GET_ARGS,cat,200,28/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,413,410,8710d0452ec15257b16b9e690d945989,REQUEST_GET_ARGS,cat,200,28/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,413,410,8710d0452ec15257b16b9e690d945989,REQUEST_GET_ARGS,cat,200,28/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...
183,4,-51.5967,-17.9714,1,24.6521,76.8835,612,609,7312ddcb0cd329b3c0799a9b8e24f080,REQUEST_GET_ARGS,cat,200,22/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,612,609,7312ddcb0cd329b3c0799a9b8e24f080,REQUEST_GET_ARGS,cat,200,22/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,612,609,7312ddcb0cd329b3c0799a9b8e24f080,REQUEST_GET_ARGS,cat,200,22/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,1,24.6521,76.8835,612,609,7312ddcb0cd329b3c0799a9b8e24f080,REQUEST_GET_ARGS,cat,200,22/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,612,609,7312ddcb0cd329b3c0799a9b8e24f080,REQUEST_GET_ARGS,cat,200,22/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,612,609,7312ddcb0cd329b3c0799a9b8e24f080,REQUEST_GET_ARGS,cat,200,22/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...
995,4,-51.5756,-17.8,1,24.7301,76.8928,169,169,e8c8a7f283bff7cabc32f667abe94ba6,REQUEST_GET_ARGS,cat,200,93/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,169,169,e8c8a7f283bff7cabc32f667abe94ba6,REQUEST_GET_ARGS,cat,200,93/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,169,169,e8c8a7f283bff7cabc32f667abe94ba6,REQUEST_GET_ARGS,cat,200,93/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,1,24.7301,76.8928,169,169,e8c8a7f283bff7cabc32f667abe94ba6,REQUEST_GET_ARGS,cat,200,93/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,169,169,e8c8a7f283bff7cabc32f667abe94ba6,REQUEST_GET_ARGS,cat,200,93/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,169,169,e8c8a7f283bff7cabc32f667abe94ba6,REQUEST_GET_ARGS,cat,200,93/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...
1198,4,-51.4622,-17.5483,1,24.7752,76.9127,776,776,4620118a990457c86f85413d44695ab5,REQUEST_GET_ARGS,cat,200,157/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/...,776,776,4620118a990457c86f85413d44695ab5,REQUEST_GET_ARGS,cat,200,157/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/...,776,776,4620118a990457c86f85413d44695ab5,REQUEST_GET_ARGS,cat,200,157/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/...,1,24.7752,76.9127,776,776,4620118a990457c86f85413d44695ab5,REQUEST_GET_ARGS,cat,200,157/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/...,776,776,4620118a990457c86f85413d44695ab5,REQUEST_GET_ARGS,cat,200,157/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/...,776,776,4620118a990457c86f85413d44695ab5,REQUEST_GET_ARGS,cat,200,157/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/...
1278,4,-50.8993,-17.3753,1,24.9232,77.2522,136,136,40f6b70dba73c27418f46878933207e6,REQUEST_GET_ARGS,cat,200,19/a/a/a/a/a/a/a,136,136,40f6b70dba73c27418f46878933207e6,REQUEST_GET_ARGS,cat,200,19/a/a/a/a/a/a/a,136,136,40f6b70dba73c27418f46878933207e6,REQUEST_GET_ARGS,cat,200,19/a/a/a/a/a/a/a,1,24.9232,77.2522,136,136,40f6b70dba73c27418f46878933207e6,REQUEST_GET_ARGS,cat,200,19/a/a/a/a/a/a/a,136,136,40f6b70dba73c27418f46878933207e6,REQUEST_GET_ARGS,cat,200,19/a/a/a/a/a/a/a,136,136,40f6b70dba73c27418f46878933207e6,REQUEST_GET_ARGS,cat,200,19/a/a/a/a/a/a/a
1396,4,-51.2993,-18.2944,1,24.5205,76.9988,430,430,43a79dd47c8c68015f865eb85e98e469,REQUEST_GET_ARGS,cat,200,20/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,430,430,43a79dd47c8c68015f865eb85e98e469,REQUEST_GET_ARGS,cat,200,20/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a/a...,

In [141]:
seventh_cluster

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
291,5,23.5986,-20.6011,1,-22.9341,21.6683,979,976,617b2abaa24b3ffb8c43da31ec31b026,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.oTdjDWUetc\';\'REQ...,302,"Referer;;echoe""ContentType:text/plainn"""";echoe...",979,976,617b2abaa24b3ffb8c43da31ec31b026,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.oTdjDWUetc\';\'REQ...,302,"Referer;;echoe""ContentType:text/plainn"""";echoe...",979,976,617b2abaa24b3ffb8c43da31ec31b026,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.oTdjDWUetc\';\'REQ...,302,"Referer;;echoe""ContentType:text/plainn"""";echoe...",1,-22.9341,21.6683,979,976,617b2abaa24b3ffb8c43da31ec31b026,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.oTdjDWUetc\';\'REQ...,302,"Referer;;echoe""ContentType:text/plainn"""";echoe...",979,976,617b2abaa24b3ffb8c43da31ec31b026,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.oTdjDWUetc\';\'REQ...,302,"Referer;;echoe""ContentType:text/plainn"""";echoe...",979,976,617b2abaa24b3ffb8c43da31ec31b026,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.oTdjDWUetc\';\'REQ...,302,"Referer;;echoe""ContentType:text/plainn"""";echoe..."
353,5,23.1873,-21.0536,1,-22.9478,21.652,134,134,a8bd4a50791562477aadb687ca0f86f1,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.1e49Ye11XD\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe...",134,134,a8bd4a50791562477aadb687ca0f86f1,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.1e49Ye11XD\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe...",134,134,a8bd4a50791562477aadb687ca0f86f1,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.1e49Ye11XD\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe...",1,-22.9478,21.652,134,134,a8bd4a50791562477aadb687ca0f86f1,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.1e49Ye11XD\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe...",134,134,a8bd4a50791562477aadb687ca0f86f1,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.1e49Ye11XD\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe...",134,134,a8bd4a50791562477aadb687ca0f86f1,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.1e49Ye11XD\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe..."
489,5,23.186,-20.497,1,-22.9297,21.67,638,637,cc0acfcf27f9fec19a5c75aa5f960513,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.zE4UqjNTqH\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe...",638,637,cc0acfcf27f9fec19a5c75aa5f960513,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.zE4UqjNTqH\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe...",638,637,cc0acfcf27f9fec19a5c75aa5f960513,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.zE4UqjNTqH\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe...",1,-22.9297,21.67,638,637,cc0acfcf27f9fec19a5c75aa5f960513,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.zE4UqjNTqH\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe...",638,637,cc0acfcf27f9fec19a5c75aa5f960513,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.zE4UqjNTqH\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe...",638,637,cc0acfcf27f9fec19a5c75aa5f960513,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.zE4UqjNTqH\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe..."
894,5,23.6575,-20.7968,1,-22.9346,21.6669,913,911,a2c85e770ccac658b3ab7cb4f05c8d33,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.YzfQRf9hCd\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe...",913,911,a2c85e770ccac658b3ab7cb4f05c8d33,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.YzfQRf9hCd\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe...",913,911,a2c85e770ccac658b3ab7cb4f05c8d33,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.YzfQRf9hCd\';\'REQ...,301,"Referer;;echoe""ContentType:text/plainn"""";echoe...",1,-22.9346,21.6669,913,911,a2c85e770ccac658b3ab7cb4f05c8d33,REQUEST_HEADERS,Referer\';\'REQUEST_HEADERS.YzfQRf9hCd\';\'REQ...,301,"Referer;;echoe""Conten

В ходе работы над поставленной задачей, мы использовали два метода кластеризации. Изначально мы получили два кластера с помощью KMeans, разделив исходные данные на потенциально опасные и безопасные, затем с помощью DBSCAN сделали кластеризацию кластера с потенциально опасными, чтобы попробовать разделить их по типам угроз. Как мы видим у нас получилось 7 разных кластеров. Кластер -1 - это шум. В кластере 0 1 мы видим printchr - это похоже на атаку (вмешательство в код). В кластерах 2 3 4 видим непонятные данные.